# Moving Beyond Linearity

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots

# StatsModel
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm

# ISLP
from ISLP import load_data
from ISLP.models import bs, ns
from ISLP. transforms import (BSpline, NaturalSpline)
from ISLP.models import (summarize, poly, ModelSpec as MS)
from ISLP.pygam import (approx_lam, degrees_of_freedom, plot as plot_gam, anova as anova_gam)

# PyGam
from pygam import (s as s_gam, l as l_gam, f as f_gam, LinearGAM, LogisticGAM)

### Non-Linear Modeling

#### Polynomial Regression & Step Functions

In [ ]:
Wage = load_data('Wage')

y = Wage['wage']

age = Wage['age']

In [ ]:
poly_age = MS([ poly('age', degree =4) ]).fit(Wage)

M = sm.OLS(y, poly_age.transform(Wage)).fit()

summarize (M)

In [ ]:
age_grid = np.linspace(age.min(),
                       age.max(),
                       100)

age_df = pd. DataFrame ({'age': age_grid })

In [ ]:
def plot_wage_fit(age_df ,basis, title):
    X = basis.transform(Wage)
    Xnew = basis.transform(age_df)
    M = sm.OLS(y, X).fit()
    preds = M. get_prediction(Xnew)
    bands = preds.conf_int(alpha =0.05)
    fig , ax = subplots (figsize =(8 ,8))
    ax.scatter(age,
               y,
               facecolor ='gray',
               alpha =0.5)
    for val, ls in zip ([ preds.predicted_mean,
                          bands [:, 0],
                          bands [:, 1]],
                        ['b','r--','r--']):
        ax.plot(age_df.values, val , ls , linewidth =3)
        ax. set_title (title, fontsize =20)
        ax. set_xlabel ('Age', fontsize =20)
        ax. set_ylabel ('Wage', fontsize =20);
        return ax

In [ ]:
plot_wage_fit(age_df,
              poly_age,
              'Degree -4 Polynomial');

In [ ]:
models = [MS([poly('age', degree=d)])
          for d in range (1, 6)]

Xs = [model. fit_transform(Wage) for model in models]
anova_lm (*[sm.OLS(y, X_).fit()
            for X_ in Xs])

In [ ]:
summarize(M)

In [ ]:
(-11.983) **2

In [ ]:
models = [MS(['education', poly('age', degree=d)])
          for d in range(1, 4)]

XEs = [model.fit_transform(Wage)
       for model in models]

anova_lm (*[ sm.OLS(y, X_).fit() for X_ in XEs ])

In [ ]:
X = poly_age.transform (Wage)
high_earn = Wage['high_earn'] = y > 250 # shorthand

glm = sm.GLM(y > 250,
             X,
             family=sm.families.Binomial())

B = glm.fit()
summarize (B)

In [ ]:
newX = poly_age.transform(age_df)

preds = B.get_prediction(newX)
bands = preds.conf_int(alpha=0.05)

In [ ]:
fig , ax = subplots (figsize=(8, 8))

rng = np.random. default_rng(0)
ax.scatter(age +
           0.2 * rng.uniform(size=y.shape[0]),
           np.where(high_earn , 0.198 , 0.002),
           fc='gray',
           marker='|')

for val, ls in zip([ preds.predicted_mean,
                     bands [:, 0],
                     bands [:, 1]],
                   ['b','r--','r--']):
    ax.plot(age_df.values, val, ls, linewidth =3)
    ax. set_title ('Degree -4 Polynomial', fontsize =20)
    ax. set_xlabel ('Age', fontsize =20)
    ax. set_ylim ([0 ,0.2])
    ax. set_ylabel ('P(Wage > 250)', fontsize =20);
    plt.grid()

In [ ]:
cut_age = pd.qcut(age, 4)

summarize (sm.OLS(y, pd.get_dummies(cut_age)).fit())

#### Splines

In [ ]:
bs_ = BSpline(internal_knots =[25 ,40 ,60], intercept =True).fit(age)

bs_age = bs_.transform(age)
bs_age.shape

In [ ]:
bs_age = MS([bs('age', internal_knots =[25 ,40 ,60])])

Xbs = bs_age.fit_transform(Wage)

M = sm.OLS(y, Xbs).fit()
summarize (M)

In [ ]:
bs_age = MS([bs('age',
                internal_knots=[25 ,40 ,60],
                name='bs(age)')])

Xbs = bs_age.fit_transform(Wage)
M = sm.OLS(y, Xbs).fit()
summarize (M)

In [ ]:
BSpline(df=6).fit(age).internal_knots_

In [ ]:
bs_age0 = MS([bs('age',
                 df=3,
                 degree =0) ]).fit(Wage)

Xbs0 = bs_age0. transform (Wage)

summarize (sm.OLS(y, Xbs0).fit())

In [ ]:
ns_age = MS([ns('age', df =5)]).fit(Wage)

M_ns = sm.OLS(y, ns_age.transform(Wage)).fit()

summarize (M_ns)

In [ ]:
plot_wage_fit (age_df,
               ns_age,
               'Natural spline, df=5');

plt.grid(True)

#### Smoothing Splines & GAMs

In [ ]:
X_age = np.asarray(age).reshape(( -1 ,1))

gam = LinearGAM (s_gam (0, lam =0.6))
gam.fit(X_age, y)

In [ ]:
fig , ax = subplots(figsize =(8, 8))

ax.scatter(age, y, facecolor ='gray', alpha =0.5)

for lam in np. logspace (-2, 6, 5):
    gam = LinearGAM (s_gam (0, lam=lam)).fit(X_age, y)
    ax.plot(age_grid,
            gam.predict( age_grid),
            label='{:.1e}'.format(lam),
            linewidth=3)
    ax.set_xlabel ('Age', fontsize =20)
    ax.set_ylabel ('Wage', fontsize =20);
    ax.legend(title='$\ lambda$');
    plt.grid()

In [ ]:
gam_opt = gam. gridsearch (X_age , y)
ax.plot(age_grid ,
gam_opt.predict( age_grid ),
label='Grid search ',
linewidth =4)
ax.legend ()
fig

In [ ]:
age_term = gam.terms[0]

lam_4 = approx_lam (X_age, age_term, 4)

age_term.lam = lam_4
degrees_of_freedom (X_age, age_term )

In [ ]:
fig , ax = subplots (figsize =(8 ,8))

ax.scatter(X_age,
           y,
           facecolor ='gray',
           alpha =0.3)

for df in [1, 3, 4, 8, 15]:
    lam = approx_lam(X_age , age_term , df +1)
    age_term.lam = lam
    gam.fit(X_age, y)

ax.plot(age_grid,
        gam.predict(age_grid),
        label='{:d}'.format(df),
        linewidth =4)

ax.set_xlabel('Age', fontsize =20)
ax.set_ylabel('Wage', fontsize =20);
ax.legend(title='Degrees of freedom ');
plt.grid()

#### Additive Models with Several Terms

In [ ]:
ns_age = NaturalSpline(df =4).fit(age)
ns_year = NaturalSpline (df =5).fit(Wage['year'])

Xs = [ns_age.transform (age),
      ns_year.transform (Wage['year']),
      pd. get_dummies(Wage['education']).values]

X_bh = np.hstack(Xs)
gam_bh = sm.OLS(y, X_bh).fit()

In [ ]:
age_grid = np.linspace(age.min(),
                       age.max(),
                       100)

X_age_bh = X_bh.copy()[:100]
X_age_bh[:] = X_bh[:].mean(0)[None, :]
X_age_bh [: ,:4] = ns_age.transform(age_grid)

preds = gam_bh.get_prediction(X_age_bh)
bounds_age = preds.conf_int(alpha=0.05)
partial_age = preds.predicted_mean
center = partial_age.mean()
partial_age -= center
bounds_age -= center

fig , ax = subplots(figsize=(8, 8))
ax.plot(age_grid, partial_age, 'b', linewidth =3)
ax.plot(age_grid, bounds_age[:,0], 'r--', linewidth =3)
ax.plot(age_grid, bounds_age[:,1], 'r--', linewidth =3)
ax. set_xlabel ('Age')
ax. set_ylabel ('Effect on wage')
ax. set_title ('Partial dependence of age on wage', fontsize =20);
plt.grid()

In [ ]:
year_grid = np.linspace (2003, 2009, 100)
year_grid = np.linspace (Wage['year'].min(), Wage['year'].max(),100)

X_year_bh = X_bh.copy()[:100]
X_year_bh[:] = X_bh[:].mean(0)[None, :]
X_year_bh[: ,4:9] = ns_year.transform(year_grid)

preds = gam_bh.get_prediction(X_year_bh)
bounds_year = preds.conf_int(alpha=0.05)
partial_year = preds.predicted_mean
center = partial_year.mean()
partial_year -= center
bounds_year -= center

fig , ax = subplots(figsize =(8, 8))
ax.plot(year_grid, partial_year, 'b', linewidth =3)
ax.plot(year_grid, bounds_year[:,0], 'r--', linewidth =3)
ax.plot(year_grid, bounds_year[:,1], 'r--', linewidth =3)
ax. set_xlabel ('Year')
ax. set_ylabel ('Effect on wage')
ax. set_title ('Partial dependence of year on wage', fontsize =20);
plt.grid()

In [ ]:
gam_full = LinearGAM(s_gam (0) +
                     s_gam (1, n_splines=7) +
                     f_gam (2, lam=0))

Xgam = np.column_stack([age,
                        Wage['year'],
                        Wage['education'].cat.codes])

gam_full = gam_full .fit(Xgam , y)

In [ ]:
fig , ax = subplots(figsize =(8 ,8))
plot_gam (gam_full, 0, ax=ax)
ax.set_xlabel('Age')
ax.set_ylabel('Effect on wage')
ax.set_title ('Partial dependence of age on wage - default lam =0.6',
              fontsize =20);
plt.grid()

In [ ]:
age_term = gam_full.terms[0]
age_term.lam = approx_lam (Xgam, age_term, df =4+1)

year_term = gam_full.terms[1]
year_term.lam = approx_lam (Xgam, year_term, df =4+1)

gam_full = gam_full.fit(Xgam, y)

In [ ]:
fig, ax = subplots(figsize =(8, 8))
plot_gam (gam_full,
          1,
          ax=ax)
ax.set_xlabel ('Year')
ax.set_ylabel ('Effect on wage')
ax.set_title ('Partial dependence of year on wage', fontsize =20)
plt.grid()

In [ ]:
fig , ax = subplots (figsize =(8, 8))

ax = plot_gam(gam_full , 2)
ax. set_xlabel('Education')
ax. set_ylabel('Effect on wage')
ax. set_title('Partial dependence of wage on education',fontsize =20);
ax. set_xticklabels(Wage['education'].cat.categories, fontsize =8);
plt.grid()

#### ANOVA Tests for Additive Models

In [ ]:
gam_0 = LinearGAM(age_term + f_gam (2, lam =0))
gam_0.fit(Xgam, y)

gam_linear = LinearGAM(age_term +
                       l_gam(1, lam =0) +
                       f_gam(2, lam =0))

gam_linear.fit(Xgam, y)

In [ ]:
anova_gam(gam_0, gam_linear, gam_full)

In [ ]:
gam_0 = LinearGAM(year_term +
                  f_gam(2, lam =0))

gam_linear = LinearGAM(l_gam(0, lam =0) +
                       year_term +
                       f_gam(2, lam =0))

gam_0.fit(Xgam, y)
gam_linear.fit(Xgam, y)

anova_gam(gam_0 , gam_linear , gam_full)

In [ ]:
gam_full.summary()

In [ ]:
Yhat = gam_full.predict(Xgam)

In [ ]:
gam_logit = LogisticGAM(age_term +
                        l_gam(1, lam =0) +
                        f_gam(2, lam =0))

gam_logit.fit(Xgam, high_earn)

In [ ]:
fig , ax = subplots(figsize =(8, 8))

ax = plot_gam(gam_logit, 2)
ax.set_xlabel('Education')
ax.set_ylabel('Effect on wage')
ax.set_title('Partial dependence of wage on education', fontsize =20);
ax.set_xticklabels(Wage['education'].cat.categories, fontsize =8);
plt.grid()

In [ ]:
pd.crosstab(Wage['high_earn'], Wage['education'])

In [ ]:
only_hs = Wage['education'] == '1. < HS Grad'

Wage_ = Wage.loc[~only_hs]
Xgam_ = np.column_stack([ Wage_['age'],
                          Wage_['year'],
                          Wage_['education'].cat.codes -1])

high_earn_ = Wage_['high_earn']

In [ ]:
gam_logit_ = LogisticGAM(age_term +
                         year_term +
                         f_gam(2, lam =0))

gam_logit_ .fit(Xgam_, high_earn_)

In [ ]:
fig , ax = subplots(figsize=(8, 8))

ax = plot_gam(gam_logit_, 2)
ax.set_xlabel('Education')
ax.set_ylabel('Effect on wage')
ax.set_title('Partial dependence of high earner status on education', fontsize =20);
ax.set_xticklabels(Wage['education']. cat. categories [1:], fontsize =8);
plt.grid()

In [ ]:
fig , ax = subplots(figsize =(8, 8))

ax = plot_gam(gam_logit_, 1)
ax.set_xlabel('Year')
ax.set_ylabel('Effect on wage')
ax.set_title('Partial dependence of high earner status on year', fontsize =20);
plt.grid()

In [ ]:
fig , ax = subplots (figsize =(8, 8))

ax = plot_gam (gam_logit_, 0)
ax.set_xlabel('Age')
ax.set_ylabel('Effect on wage')
ax.set_title('Partial dependence of high earner status on age', fontsize =20);
plt.grid()

#### Local Regression

In [ ]:
lowess = sm.nonparametric.lowess

fig , ax = subplots(figsize =(8, 8))
ax.scatter(age, y, facecolor ='gray', alpha =0.5)

for span in [0.2, 0.5]:
    fitted = lowess(y,
                    age,
                    frac=span,
                    xvals=age_grid)
    ax.plot(age_grid,
            fitted,
            label='{:.1f}'.format(span),
            linewidth=4)
    ax. set_xlabel('Age', fontsize =20)
    ax. set_ylabel('Wage', fontsize =20);
    ax.legend(title='span', fontsize =15);
    plt.grid()